In [ ]:
import boto3
import json
from jinja2 import Environment, FileSystemLoader
import sys
from pathlib import Path
import base64
import os

In [ ]:
input_image_dir = os.path.abspath(os.path.join(os.getcwd(),"../../","content/images"))
input_image_name = "table_example.png"
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
input_text="Describe this image in detail. Focus on identifying entities and thier relationships within the image"

In [ ]:
output_dir = os.path.abspath(os.path.join(os.getcwd(),"../..","model-results"))
output_dir

In [ ]:
template_filename = "table-test"

In [ ]:
template_dir = os.path.abspath(os.path.join(os.getcwd(),"../..","prompts-templates"))
template_dir
environment = Environment(loader=FileSystemLoader(template_dir))
template = environment.get_template("prompt-without-ontology-system.txt")
system_input= template.render()

In [ ]:
with open(os.path.join(input_image_dir,input_image_name),"rb") as f:
    content_image = base64.b64encode(f.read()).decode('utf8')

In [ ]:
bedrock_runtime= boto3.client("bedrock-runtime",region_name="us-east-1")

In [ ]:
message={"role": "user",
             "content": [
                {"type": "text", "text": "<input>"},
                {"type": "image", "source": {"type": "base64",
                    "media_type": "image/png", "data": content_image}},
                {"type": "text", "text": "</input>"}
                ]}

In [ ]:
messages = [message]

In [ ]:
body=json.dumps(
{
    'messages':messages,
    'anthropic_version':'bedrock-2023-05-31',
    'top_p':0.5,
    'temperature':0.5,
    'top_k':5,
    'max_tokens':2000,
    'system':system_input
})

In [ ]:
response = bedrock_runtime.invoke_model(
    modelId='anthropic.claude-3-sonnet-20240229-v1:0',
    contentType='application/json',
    accept='application/json',
    body=body
)

In [ ]:
response_body = json.loads(response['body'].read())

In [ ]:
response_body

In [ ]:
output_name = "test"

In [ ]:
full_output_name = f"{model_id}-{template_filename.split('.txt')[0]}-{output_name}"

In [ ]:
same_name_count = int(len([f for f in os.listdir(output_dir) if full_output_name in f])/2)

In [ ]:
file_count = "" if same_name_count ==0 else f'-{same_name_count}'

In [ ]:
with open(os.path.join(output_dir,f"{full_output_name}{file_count}.json"),"w+") as f:
    json.dump(response_body,f)

In [ ]:
with open(os.path.join(output_dir,f"{full_output_name}-graph{file_count}.json"),"w+") as f:
    json.dump(json.loads(response_body["content"][0]["text"].split("```")[1].split("json")[1]),f)